# YOLOを用いた学習

[こちらの記事](https://farml1.com/yolov7/)を参考にした。

In [1]:
import os
from glob import glob
from collections import defaultdict
import random
import shutil

## 定数定義

In [2]:
DATA_PATH = os.path.join('..','data')
IMAGE_PATH = os.path.join(DATA_PATH,'images')
LABEL_PATH = os.path.join(DATA_PATH,'labels')

CUSTOM_DATASET_PATH = os.path.join(DATA_PATH,'custom_dataset')
CUSTOM_IMAGE_PATH = os.path.join(CUSTOM_DATASET_PATH,'images')
CUSTOM_LABEL_PATH = os.path.join(CUSTOM_DATASET_PATH,'labels')
TRAIN_IMAGE_PATH = os.path.join(CUSTOM_IMAGE_PATH,'train')
VALID_IMAGE_PATH = os.path.join(CUSTOM_IMAGE_PATH,'valid')
TRAIN_LABEL_PATH = os.path.join(CUSTOM_LABEL_PATH,'train')
VALID_LABEL_PATH = os.path.join(CUSTOM_LABEL_PATH,'valid')

## 画像とラベルファイルの整合性の確認  
いくつかのラベルや画像を除去しているので、画像とラベルファイルが対応するように整える

In [3]:
imgAry = set([os.path.basename(img_path) for img_path in glob(os.path.join(IMAGE_PATH,'*.png'))])
lblAry = set([os.path.basename(lbl_path) for lbl_path in glob(os.path.join(LABEL_PATH,'*.txt'))])
len(imgAry), len(lblAry)

(200, 201)

In [4]:
mx = len(imgAry)
for i in range(1, mx+1):
    file_num = str(i).zfill(4)
    img_name = f"{file_num}.png"
    lbl_name = f"{file_num}.txt"
    if img_name in imgAry and lbl_name in lblAry:
        continue
    imgAry.discard(img_name)
    lblAry.discard(lbl_name)

In [5]:
len(imgAry), len(lblAry)

(200, 201)

画像とラベルファイルの枚数が一致したことを確認(classes.txtの影響で1枚ずれる)。

## 不要な画像の除去

In [6]:
for img_path in glob(os.path.join(IMAGE_PATH,'*.png')):
    img_name = os.path.basename(img_path)
    lbl_name = os.path.splitext(img_name)[0] + '.txt'
    if lbl_name not in lblAry:
        try:
            os.remove(os.path.join(IMAGE_PATH,img_name))
        except FileNotFoundError:
            continue

## クラス数の確認  
データが不均衡でないかを見てみる。

In [7]:
dict = defaultdict(int)
for lbl_path in glob(os.path.join(LABEL_PATH,'*.txt')):
    if 'class' in lbl_path:
        continue
    with open(lbl_path, 'r') as rf:
        lines = rf.readlines()
        for line in lines:
            line = line.rstrip('\n')
            num = int(line.split()[0])
            dict[num] += 1

In [8]:
dict

defaultdict(int, {4: 132, 2: 132, 0: 125, 1: 130, 3: 129})

以外にも悪くない感じ。割と均等にラベル付けができている。

## ランダムに訓練データと評価データに分ける  
train:valid = 7:3

In [9]:
# 変数リセット
imgAry = sorted([img_path for img_path in glob(os.path.join(IMAGE_PATH,'*.png'))])
lblAry = sorted([lbl_path for lbl_path in glob(os.path.join(LABEL_PATH,'*.txt'))])
lblAry.remove(os.path.join(LABEL_PATH,'classes.txt')) # classes.txtを除く

In [10]:
os.makedirs(TRAIN_IMAGE_PATH, exist_ok=True)
os.makedirs(TRAIN_LABEL_PATH, exist_ok=True)
os.makedirs(VALID_IMAGE_PATH, exist_ok=True)
os.makedirs(VALID_LABEL_PATH, exist_ok=True)

In [11]:
dataset = []
for img_path, lbl_path in zip(imgAry, lblAry):
    dataset.append([img_path, lbl_path])

In [12]:
n = int(7*len(imgAry)/10) #訓練データが7割
train = random.sample(dataset, n)
valid = [i for i in dataset if i not in train]
len(train), len(valid)

(140, 60)

In [13]:
# データのコピー
def copy_files(target_dataset, img_folder, lbl_folder):
    for img_path, lbl_path in target_dataset:
        mv_img_path = os.path.join(img_folder,os.path.basename(img_path))
        mv_lbl_path = os.path.join(lbl_folder,os.path.basename(lbl_path))
        shutil.copy(img_path, mv_img_path)
        shutil.copy(lbl_path, mv_lbl_path)

In [14]:
copy_files(train, TRAIN_IMAGE_PATH, TRAIN_LABEL_PATH)

In [15]:
copy_files(valid, VALID_IMAGE_PATH, VALID_LABEL_PATH)

## 学習に使用するyamlファイルを作成

~~手動で`yolv7/data/coco.yaml`を変更する。分類するクラス数とそのクラス名称を変更するだけ(クラス名称の順番はclasses.txtと同じ)。~~  
別に`dataset.yaml`という学習ファイルを作成した。

## YOLOによる学習

~~ターミナルで環境を整えつつ、`train.py`を実行するだけ。~~  
結局、GPUが無料で簡単に使えるgoogle colabにて動作を実行した。